# Dummy Release Report
# 2019.06 Release Codename


## Table of Contents
* [Lava Test Results](#lava_test_results)
* [Second Item](#second_item)
* [Third Item](#third_item)
* [Fourth Item](#fourth_item)
* [Fifth Item](#fifth_item)
* [Sixth Item](#sixth_item)

## Sample Markdown

## Executive Summary	 	 	 	
your exective summary goes here

## Hardware Platforms
Hardware platforms goes here:
* HW1: 
* HW2: 

# Release Highlights
## Baseline versions
Release highlights goes here:
* first
* second


#### Lava Test Results
<a id="lava_test_results"></a>

In [ ]:
import requests   
import json
from json2html import *
from IPython.display import display, HTML 
from datetime import datetime 
  
start_time = datetime.now()  

def query_rest_api(url,token=None):  
    result = ''  
    if token:  
        token_str = "token {}".format(token)  
        headers = {"Authorization": token_str}  
        result = requests.get(url, headers=headers) 
    else:  
        result = requests.get(url)   
          
    result.raise_for_status()              
    return result   


def make_new_colgrp(soup, width1, width2): 
    col1 = soup.new_tag('col') 
    col2 = soup.new_tag('col') 
    col1['width'] = width1  
    col2['width'] = width2 
    colgrp = soup.new_tag('colgroup') 
    colgrp.append(col1) 
    colgrp.append(col2) 
    return colgrp 


def process_html(html):    
    from bs4 import BeautifulSoup as bs
    soup = bs(html)
    head = soup.new_tag('head')
    link = soup.new_tag('link',
                        rel="stylesheet",
                        href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css")
    jquery = soup.new_tag('script',
                          src="https://ajax.googleapis.com/ajax/libs/jquery/3.4.1/jquery.min.js")
    bootstrap = soup.new_tag('script',
                             src="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/js/bootstrap.min.js")
    head.append(link)
    head.append(jquery)
    head.append(bootstrap)
    soup.html.insert(0, head)
    soup.ul.unwrap()     
        
    for li in soup.body.find_all('li'):  
        li.unwrap()
    tbl_class = 'table-sm table-bordered'
    tbl_style = 'width:100%'
    all_style = 'text-align: left;' 
    for th in soup.body.find_all('th', string='Testsuites'):         
        th.next_sibling.table['class'] = tbl_class
        th.next_sibling.table['style'] = tbl_style
        th.next_sibling.table.insert(0, make_new_colgrp(soup, "20%", "80%"))
        for tbl in th.next_sibling.table.find_all('table'): 
            tbl['class'] = tbl_class 
            tbl['style'] = tbl_style
            tbl.insert(0, make_new_colgrp(soup, "80%", "20%")) 
    for th in soup.body.find_all('th', string='Metric Testsuites'):          
        th.next_sibling.table['class'] = tbl_class
        th.next_sibling.table['style'] = tbl_style
        th.next_sibling.table.insert(0, make_new_colgrp(soup, "20%", "80%"))
        for tbl in th.next_sibling.table.find_all('table'):  
            tbl['class'] = tbl_class
            tbl['style'] = tbl_style
            tbl.insert(0, make_new_colgrp(soup, "80%", "20%")) 
    for tbl1 in soup.body.find_all('table', recursive=False):
        tbl1['class'] = tbl_class
        tbl['style'] = tbl_style
        tbl1.insert(0, make_new_colgrp(soup, "10%", "90%"))  
    for td in soup.find_all('td'):
        td['style'] = all_style
        if td.string == 'fail':
            td['class'] = 'bg-danger'
        if td.string == 'skip':
            td['class'] = 'bg-warning'
        if td.string == 'xfail':
            td['class'] = 'bg-info'
    for th in soup.find_all('th'):
        th['style'] = all_style
    return str(soup)

base_url = "http://qa-reports.linaro.org" #"http://127.0.0.1:8000"
base_api_url = base_url + "/api/testruns/?" #"http://127.0.0.1:8000/api/testruns/?"   
project_filter = "build__project__slug={}"   
build_filter = "&build__version={}"    
limit="&limit=200"   
project_slug = "schneider" #"linux-stable-rc-4.14-oe" 
build_version_slug = "267" #"v4.14.74" 
filtered_url = "%s%s%s%s" % (base_api_url, project_filter, build_filter, limit)   
url = filtered_url.format(project_slug, build_version_slug)   
lava_api_urlfrag = "/api/v0.1/jobs/"
  
qa_response = ''   
test_runs = []
my_token = "YOUR SQUAD TOKEN GOES HERE"
qa_response = query_rest_api(url, my_token).json()
buid_api_url = qa_response['results'][0]['build']
build_api_response = query_rest_api(buid_api_url, my_token).json()
project_api_url = build_api_response['project']
project_api_response = query_rest_api(project_api_url, my_token).json()
project_slug = project_api_response['slug']
group_api_url = project_api_response['group']
group_api_response = query_rest_api(group_api_url, my_token).json()
group_slug = query_rest_api(group_api_url, my_token).json()['slug']
testrun_abslt_url = "{}/{}/{}/build/{}".format(base_url,group_slug,project_slug,build_version_slug)

while True:            
    url = qa_response['next']   
    testruns = qa_response['results']
    testruns_cnt = 0    
    for tr in testruns:
        tests_response = query_rest_api(tr['tests'], my_token).json()
        metrics_response = query_rest_api(tr['metrics'], my_token).json()
        if tests_response['count'] == 0 and metrics_response['count'] == 0:            
                continue
        env_dict = {'Environment': query_rest_api(tr['environment'], my_token).json()['slug']}                              
        lavajob_api_url = tr['job_url'].rsplit('/', 3)[0] + lava_api_urlfrag + tr['job_id']
        job_description = query_rest_api(lavajob_api_url, None).json()['description']
        env_dict['TestRun'] = "<a href='{0}'>{1}</a>".format(tr['job_url'], job_description)          
        if tests_response['results']:           
            env_dict['Testsuites']= {}            
            env_dict['Tests Total'] = {'All': tests_response['count']}
            env_dict['Tests Total']['Pass'] = 0
            env_dict['Tests Total']['Fail'] = 0
            env_dict['Tests Total']['Skip'] = 0
            env_dict['Tests Total']['Xfail'] = 0
            while True:
                tests_next = tests_response['next']            
                tests = tests_response['results']                  
                for test in tests:
                    split_test = test['name'].split('/')   
                    suite_name = split_test[0]
                    test_name = split_test[1]
                    suite_name = "<a href='{0}/testrun/{1}/suite/{2}/tests/'>{2}</a>".format(testrun_abslt_url,
                                                                                         tr['job_id'],suite_name)  
                    if not suite_name in env_dict['Testsuites'].keys():
                        env_dict['Testsuites'][suite_name] = {}
                    env_dict['Testsuites'][suite_name][test_name] = test['status']                    
                    if test['status'] == 'pass':
                        env_dict['Tests Total']['Pass'] += 1
                    if test['status'] == 'fail':
                        env_dict['Tests Total']['Fail'] += 1
                    if test['status'] == 'xfail':
                        env_dict['Tests Total']['Xfail'] += 1
                    if test['status'] == 'skip':
                        env_dict['Tests Total']['Skip'] += 1                
                if tests_next:
                    tests_response = query_rest_api(tests_next, my_token).json()
                    continue
                else:
                    break        
        if metrics_response['results']:
            env_dict['Metric Testsuites']= {}
            env_dict['Metrics Total'] = metrics_response['count']
            while True:
                metrics_next = metrics_response['next']                                     
                metrics = metrics_response['results']   
                for metric in metrics:
                    split_metric = metric['name'].split('/')
                    suite_name = split_metric[0]
                    test_name = split_metric[1]
                    suite_name = "<a href='{0}/testrun/{1}/suite/{2}/metrics/'>{2}</a>".format(testrun_abslt_url,
                                                                                         tr['job_id'],suite_name)  
                    if not suite_name in env_dict['Metric Testsuites'].keys():
                        env_dict['Metric Testsuites'][suite_name] = {}
                    env_dict['Metric Testsuites'][suite_name][test_name] = metric['result']            
                if metrics_next:
                    metrics_response = query_rest_api(metrics_next, my_token).json()
                    continue
                else:
                    break
        test_runs.append(env_dict)              
    if url:   
        qa_response = query_rest_api(url, my_token).json()  
    else:   
        break    
#print(datetime.now()-start_time)

html_ = json2html.convert(json=json.dumps(test_runs),escape=False)

display(HTML(process_html(html_)))